In [1]:
import pandas as pd



## Extract Features 


In [30]:
# Input ข้อมูล
order = pd.read_csv(r'/Users/satayu.c/Downloads/20230720_FPS_DATA/FPS_Order.csv')
order_item = pd.read_csv(r'/Users/satayu.c/Downloads/20230720_FPS_DATA/FPS_Order_Item-001.csv')
#อันนี้จะเป็นการนำ Item ออกมา (ข้อมูล Item 20230822.xlsx)
item_list = pd.read_csv(r'/Users/satayu.c/Downloads/Foodpassion_Item.csv',dtype={'ItemCode':str})
custome = pd.read_csv(r'/Users/satayu.c/Downloads/20230720_FPS_DATA/FPS_Customer.csv')
segment = pd.read_csv(r'/Users/satayu.c/Downloads/20230720_FPS_DATA/FPS_Segmentation.csv')


/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (11,12,13,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (26,40,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.

In [31]:
#จัดการข้อมูล Order
order['orderDate'] = pd.to_datetime(order['orderDate'])
order = order.sort_values(by=['customers','orderDate'])
order = order[['cx_Name','customers','cx_channel','branch','orderDate','pointBefore','pointEarn','pointUse']]

#จัดการข้อมูล Order_Item จัดเข้ากับ Category ที่ทาง FP ส่งมา
order_item['orderDate'] = pd.to_datetime(order_item['orderDate'])
item_list_one=item_list[['Categories','ItemCode','LocalDesc']]
item_list_two = item_list[['Categories','ItemCodeBBS','LocalDesc']]
order_item = order_item.merge(item_list_one,left_on='cx_productId',right_on='ItemCode',how='left')
order_item = order_item.merge(item_list_two,left_on='cx_productId',right_on='ItemCodeBBS',how='left')

In [32]:
order_item_final = order_item[~(order_item['Categories_x'].isna()) | ~(order_item['Categories_y'].isna())]


In [33]:
order_item_final['FinalCat']='-'
order_item_final.loc[(~(order_item['Categories_x'].isnull())),'FinalCat']= order_item_final['Categories_x']
order_item_final.loc[(~(order_item['Categories_y'].isnull())),'FinalCat']= order_item_final['Categories_y']

<ipython-input-33-f9df340f15a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_item_final['FinalCat']='-'
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [34]:
del order_item_final['Categories_x']
del order_item_final['ItemCode']
del order_item_final['LocalDesc_x']
del order_item_final['Categories_y']
del order_item_final['ItemCodeBBS']
del order_item_final['LocalDesc_y']

In [35]:
order_item_final = order_item_final[ (order_item_final['FinalCat']=='BUFFET') | (order_item_final['FinalCat']=='ALACART') | (order_item_final['FinalCat']=='BEVERAGE') | (order_item_final['FinalCat']=='SINGLE DISH') | (order_item_final['FinalCat']=='COMBO') | (order_item_final['FinalCat']=='SALAD&APPETIZER') | (order_item_final['FinalCat']=='SET')| (order_item_final['FinalCat']=='DESSERT')]
order_item_price = order_item_final.groupby(['order','customers','orderDate'])['cx_totalPrice'].sum().reset_index()
order_item_itemtype = pd.pivot_table(order_item_final, index= ['order','customers','orderDate'], columns = ['FinalCat'], values=['cx_totalPrice'], aggfunc = 'count' )
order_item_itemtype=order_item_itemtype.reset_index()
order_item_itemtype = order_item_itemtype.fillna(0)
order_item_itemtype.columns = [' '.join(col).strip() for col in order_item_itemtype.columns.values]

In [37]:
custome = custome.sort_values(by=['cx_member','lastUpdate'],ascending=False)
custome = custome[custome['pdpaConsentStatus'] == True]
custome=custome[['memberIndex','cx_member','cx_category','cx_gender','cx_birthDate']]
custome['cx_birthDate'] = pd.to_datetime(custome['cx_birthDate'],format = '%Y-%m-%d',errors = 'coerce')
custome['cx_member'] = custome['cx_member'].astype(int)

In [62]:
order_merge = order.merge(order_item_price,on=['customers','orderDate'],how='left')
order_merge = order_merge.merge(order_item_itemtype,on=['customers','orderDate'],how='left')



In [66]:
del order_merge['cx_Name']



In [67]:
order_merge.loc[order_merge['cx_channel'] != 'Delivery','cx_channel'] = 'Dine In'
order_merge = order_merge.reset_index()
order_merge['orderDate'] = pd.to_datetime(order_merge['orderDate'])
order_merge = order_merge.sort_values(by = ['customers','orderDate'])
order_merge['previousDate'] = order_merge.groupby('customers')['orderDate'].shift()
order_merge['previousDate'] = order_merge['previousDate'].fillna(min(order_merge['orderDate']))
order_merge['DaysFromPreviousBuy'] = (order_merge['orderDate'] - order_merge['previousDate']).dt.days
order_merge['isNext'] = order_merge.groupby('customers')['orderDate'].shift(-1)
order_merge['nextDate'] = order_merge.groupby('customers')['orderDate'].shift(-1)
order_merge['nextDate'] = order_merge['nextDate'].fillna(max(order_merge['orderDate']))
order_merge['Recency'] = (order_merge['nextDate'] - order_merge['orderDate']).dt.days

In [68]:
#Defined which one is coming back or Not
order_merge['isNext_180'] = 'No'
order_merge['isNext_90'] = 'No'
order_merge['isNext_60'] = 'No'
order_merge['isNext_30'] = 'No'
order_merge.loc[(~(order_merge['isNext'].isnull())) & (order_merge['Recency'] <= 180),'isNext_180']='Yes'
order_merge.loc[(~(order_merge['isNext'].isnull())) & (order_merge['Recency'] <= 90),'isNext_90']='Yes'
order_merge.loc[(~(order_merge['isNext'].isnull())) & (order_merge['Recency'] <= 60),'isNext_60']='Yes'
order_merge.loc[(~(order_merge['isNext'].isnull())) & (order_merge['Recency'] <= 30),'isNext_30']='Yes'


In [69]:
order_merge_wseg = order_merge.merge(custome,left_on='customers',right_on='cx_member',how='left')
order_merge_wseg = order_merge_wseg.drop_duplicates(subset=['customers','order_x','orderDate','branch'])
order_merge_wseg = order_merge_wseg.dropna()
order_merge_wseg = order_merge_wseg.reset_index()
del order_merge_wseg['index']
order_merge_wseg = order_merge_wseg[(order_merge_wseg['cx_birthDate'].dt.year>=1931) & (order_merge_wseg['cx_birthDate'].dt.year<=2020)]
order_merge_wseg = order_merge_wseg.reset_index()
del order_merge_wseg['index']

In [70]:
order_merge_wseg['Age'] = (order_merge_wseg['orderDate'] - order_merge_wseg['cx_birthDate']).dt.days / 365.25

In [71]:
order_merge_wseg['Time'] = order_merge_wseg.groupby('customers').cumcount()+1



In [73]:
segment = segment[['customers','totalBrandTotalChannelSegment','barBqPlazaDeliveryHatoSegment','barBqPlazaDineinThOtherSegment','monthStamp','yearStamp']]

In [74]:
order_merge_wseg['orderYear'] = order_merge_wseg['orderDate'].dt.year
order_merge_wseg['orderMonth'] = order_merge_wseg['orderDate'].dt.month

In [75]:
order_merge_wseg = order_merge_wseg.merge(segment,left_on=['customers','orderMonth','orderYear'],right_on=['customers','monthStamp','yearStamp'],how='left')

In [76]:
order_merge_wseg = order_merge_wseg.rename({'barBqPlazaDeliveryHatoSegment':'BBQPlazaDeli','barBqPlazaDineinThOtherSegment':'BBQPlazaDineIn','totalBrandTotalChannelSegment':'TotalSegment'},axis='columns')

In [77]:
del order_merge_wseg['monthStamp']
del order_merge_wseg['yearStamp']


In [78]:
order_merge_wseg['TotalSegment'] = order_merge_wseg['TotalSegment'].fillna('Unknown')
order_merge_wseg['BBQPlazaDeli'] = order_merge_wseg['BBQPlazaDeli'].fillna('Unknown')
order_merge_wseg['BBQPlazaDineIn'] = order_merge_wseg['BBQPlazaDineIn'].fillna('Unknown')

In [79]:
order_merge_wseg.columns



Index(['level_0', 'customers', 'cx_channel', 'branch', 'orderDate',
       'pointBefore', 'pointEarn', 'pointUse', 'order_x', 'cx_totalPrice',
       'order_y', 'cx_totalPrice ALACART', 'cx_totalPrice BEVERAGE',
       'cx_totalPrice BUFFET', 'cx_totalPrice COMBO', 'cx_totalPrice DESSERT',
       'cx_totalPrice SALAD&APPETIZER', 'cx_totalPrice SET',
       'cx_totalPrice SINGLE DISH', 'previousDate', 'DaysFromPreviousBuy',
       'isNext', 'nextDate', 'Recency', 'isNext_180', 'isNext_90', 'isNext_60',
       'isNext_30', 'memberIndex', 'cx_member', 'cx_category', 'cx_gender',
       'cx_birthDate', 'Age', 'Time', 'orderYear', 'orderMonth',
       'TotalSegment', 'BBQPlazaDeli', 'BBQPlazaDineIn'],
      dtype='object')

In [81]:
order_ppst = order_merge_wseg[['branch','cx_channel','pointBefore','pointEarn','pointUse','cx_totalPrice','DaysFromPreviousBuy','Time','cx_category','cx_gender','Age','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn','cx_totalPrice ALACART', 'cx_totalPrice BEVERAGE',
       'cx_totalPrice BUFFET', 'cx_totalPrice COMBO', 'cx_totalPrice DESSERT',
       'cx_totalPrice SALAD&APPETIZER', 'cx_totalPrice SET',
       'cx_totalPrice SINGLE DISH','isNext_180','isNext_90','isNext_60','isNext_30']]

## Modeling


In [82]:
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)


In [83]:
order_ppst['y_180'] = 0
order_ppst['y_90'] = 0
order_ppst['y_60'] = 0
order_ppst['y_30'] = 0
order_ppst.loc[order_ppst['isNext_180']=='Yes','y_180']=1
order_ppst.loc[order_ppst['isNext_90']=='Yes','y_90']=1
order_ppst.loc[order_ppst['isNext_60']=='Yes','y_60']=1
order_ppst.loc[order_ppst['isNext_30']=='Yes','y_30']=1


<ipython-input-83-c67da7c5dccc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_ppst['y_180'] = 0
<ipython-input-83-c67da7c5dccc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_ppst['y_90'] = 0
<ipython-input-83-c67da7c5dccc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [84]:
order_ppst_final = order_ppst.copy()


In [85]:
del order_ppst_final['isNext_180']
del order_ppst_final['isNext_90']
del order_ppst_final['isNext_60']
del order_ppst_final['isNext_30']



In [86]:
import statsmodels.api as sm
import numpy as np
from sklearn import preprocessing

# For 180 Days

In [87]:
import numpy as np
order_ppst_final180 = order_ppst_final.copy()
del order_ppst_final180['y_90']
del order_ppst_final180['y_60']
del order_ppst_final180['y_30']
X = order_ppst_final180.loc[:, order_ppst_final180.columns != 'y_180']
y = order_ppst_final180.loc[:, order_ppst_final180.columns == 'y_180']
le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train180, X_test180, y_train180, y_test180 = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train180.columns
os_data_X,os_data_y=os.fit_resample(X_train180, y_train180)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y_180'])
# we can Check the numbers of our data
#print("length of oversampled data is ",len(os_data_X))
#print("Number of not coming next is",len(os_data_y[os_data_y['y_180']==0]))
#print("Number of coming next is",len(os_data_y[os_data_y['y_180']==1]))
#print("Proportion of not coming next is ",len(os_data_y[os_data_y['y_180']==0])/len(os_data_X))
#print("Proportion of coming next is",len(os_data_y[os_data_y['y_180']==1])/len(os_data_X))


os_data_X['Age'] = os_data_X['Age'].astype(int)

X_180=os_data_X
y_180=pd.DataFrame(os_data_y['y_180'])

y_180 = np.asarray(y_180)
X_180 = np.asarray(X_180)

min_max_scaler = preprocessing.StandardScaler()
x180_scaled = min_max_scaler.fit_transform(X_180)

/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [88]:
order_ppst_final90 = order_ppst_final.copy()
del order_ppst_final90['y_180']
del order_ppst_final90['y_60']
del order_ppst_final90['y_30']
X = order_ppst_final90.loc[:, order_ppst_final90.columns != 'y_90']
y = order_ppst_final90.loc[:, order_ppst_final90.columns == 'y_90']
le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train90, X_test90, y_train90, y_test90 = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train90.columns
os_data_X,os_data_y=os.fit_resample(X_train90, y_train90)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y_90'])
# we can Check the numbers of our data
#print("length of oversampled data is ",len(os_data_X))
#print("Number of not coming next is",len(os_data_y[os_data_y['y_90']==0]))
#print("Number of coming next is",len(os_data_y[os_data_y['y_90']==1]))
#print("Proportion of not coming next is ",len(os_data_y[os_data_y['y_90']==0])/len(os_data_X))
#print("Proportion of coming next is",len(os_data_y[os_data_y['y_90']==1])/len(os_data_X))


os_data_X['Age'] = os_data_X['Age'].astype(int)

X_90=os_data_X
y_90=pd.DataFrame(os_data_y['y_90'])

y_90 = np.asarray(y_90)
X_90 = np.asarray(X_90)

min_max_scaler = preprocessing.StandardScaler()
x90_scaled = min_max_scaler.fit_transform(X_90)

/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [89]:
order_ppst_final60 = order_ppst_final.copy()
del order_ppst_final60['y_180']
del order_ppst_final60['y_90']
del order_ppst_final60['y_30']
X = order_ppst_final60.loc[:, order_ppst_final60.columns != 'y_60']
y = order_ppst_final60.loc[:, order_ppst_final60.columns == 'y_60']
le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train60, X_test60, y_train60, y_test60 = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train60.columns
os_data_X,os_data_y=os.fit_resample(X_train60, y_train60)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y_60'])
# we can Check the numbers of our data
#print("length of oversampled data is ",len(os_data_X))
#print("Number of not coming next is",len(os_data_y[os_data_y['y_60']==0]))
#print("Number of coming next is",len(os_data_y[os_data_y['y_60']==1]))
#print("Proportion of not coming next is ",len(os_data_y[os_data_y['y_60']==0])/len(os_data_X))
#print("Proportion of coming next is",len(os_data_y[os_data_y['y_60']==1])/len(os_data_X))


os_data_X['Age'] = os_data_X['Age'].astype(int)

X_60=os_data_X
y_60=pd.DataFrame(os_data_y['y_60'])

y_60 = np.asarray(y_60)
X_60 = np.asarray(X_60)

min_max_scaler = preprocessing.StandardScaler()
x60_scaled = min_max_scaler.fit_transform(X_60)

/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [90]:
order_ppst_final30 = order_ppst_final.copy()
del order_ppst_final30['y_180']
del order_ppst_final30['y_90']
del order_ppst_final30['y_60']
X = order_ppst_final30.loc[:, order_ppst_final30.columns != 'y_30']
y = order_ppst_final30.loc[:, order_ppst_final30.columns == 'y_30']
le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train30, X_test30, y_train30, y_test30 = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train30.columns
os_data_X,os_data_y=os.fit_resample(X_train30, y_train30)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y_30'])
# we can Check the numbers of our data
#print("length of oversampled data is ",len(os_data_X))
#print("Number of not coming next is",len(os_data_y[os_data_y['y_30']==0]))
#print("Number of coming next is",len(os_data_y[os_data_y['y_30']==1]))
#print("Proportion of not coming next is ",len(os_data_y[os_data_y['y_30']==0])/len(os_data_X))
#print("Proportion of coming next is",len(os_data_y[os_data_y['y_30']==1])/len(os_data_X))

os_data_X['Age'] = os_data_X['Age'].astype(int)

X_30=os_data_X
y_30=pd.DataFrame(os_data_y['y_30'])

y_30 = np.asarray(y_30)
X_30 = np.asarray(X_30)

min_max_scaler = preprocessing.StandardScaler()
x30_scaled = min_max_scaler.fit_transform(X_30)

/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [91]:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X_train180, X_test180, y_train180, y_test180 = train_test_split(x180_scaled, y_180, test_size=0.3, random_state=0)
logreg180 = LogisticRegression()
logreg180.fit(X_train180, y_train180)

X_train90, X_test90, y_train90, y_test90 = train_test_split(x90_scaled, y_90, test_size=0.3, random_state=0)
logreg90 = LogisticRegression()
logreg90.fit(X_train90, y_train90)

X_train60, X_test60, y_train60, y_test60 = train_test_split(x60_scaled, y_60, test_size=0.3, random_state=0)
logreg60 = LogisticRegression()
logreg60.fit(X_train60, y_train60)

X_train30, X_test30, y_train30, y_test30 = train_test_split(x30_scaled, y_30, test_size=0.3, random_state=0)
logreg30 = LogisticRegression()
logreg30.fit(X_train30, y_train30)


/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d arr

LogisticRegression()

In [93]:
order_ppst_final



,branch,cx_channel,pointBefore,pointEarn,pointUse,cx_totalPrice,DaysFromPreviousBuy,Time,cx_category,cx_gender,...,cx_totalPrice BUFFET,cx_totalPrice COMBO,cx_totalPrice DESSERT,cx_totalPrice SALAD&APPETIZER,cx_totalPrice SET,cx_totalPrice SINGLE DISH,y_180,y_90,y_60,y_30
0,281,Dine In,97,62,0,799.0,45,1,GON GANG FRIEND,M,...,0.0,1.0,0.0,0.0,0.0,0.0,1,1,1,0
1,281,Dine In,159,44,0,1273.0,37,2,GON GANG FRIEND,M,...,3.0,1.0,0.0,1.0,0.0,0.0,1,1,1,0
2,281,Dine In,159,44,0,1273.0,37,2,GON GANG FRIEND,M,...,3.0,1.0,0.0,1.0,0.0,0.0,1,1,1,0
3,108,Dine In,185,44,0,1119.0,7,1,GON GANG BUDDY,M,...,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0,0
4,230,Dine In,229,62,0,1711.0,65,2,GON GANG BUDDY,M,...,0.0,0.0,5.0,6.0,1.0,1.0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820607,255,Dine In,0,86,0,1446.0,214,1,GON GANG FRIEND,O,...,0.0,1.0,0.0,2.0,0.0,0.0,1,1,1,1
820608,116,Dine In,0,132,0,2392.0,214,1,GON GANG FRIEND,O,...,0.0,2.0,0.0,0.0,0.0,0.0,1,1,1,1
820609,228,Dine In,0,35,0,1048.0,214,1,GON GANG FRIEND,O,...,0.0,1.0,0.0,2.0,0.0,0.0,1,1,1,1
820610,279,Dine In,0,258,0,3886.0,214,1,GON GANG FRIEND,O,...,4.0,3.0,0.0,3.0,0.0,1.0,1,1,1,1


In [94]:
#Training with real-data

X180_real = order_ppst_final180.loc[:, (order_ppst_final180.columns != 'y_180')]
y180_real = order_ppst_final180.loc[:, order_ppst_final180.columns == 'y_180']

le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X180_real[categorical_cols] = X180_real[categorical_cols].apply(lambda col: le.fit_transform(col))

x180_real_scaled = min_max_scaler.fit_transform(X180_real)




X90_real = order_ppst_final90.loc[:, (order_ppst_final90.columns != 'y_90')]
y90_real = order_ppst_final90.loc[:, order_ppst_final90.columns == 'y_90']

le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X90_real[categorical_cols] = X90_real[categorical_cols].apply(lambda col: le.fit_transform(col))

x90_real_scaled = min_max_scaler.fit_transform(X90_real)




X60_real = order_ppst_final60.loc[:, (order_ppst_final60.columns != 'y_60')]
y60_real = order_ppst_final60.loc[:, order_ppst_final60.columns == 'y_60']

le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X60_real[categorical_cols] = X60_real[categorical_cols].apply(lambda col: le.fit_transform(col))

x60_real_scaled = min_max_scaler.fit_transform(X60_real)




X30_real = order_ppst_final30.loc[:, (order_ppst_final30.columns != 'y_30')]
y30_real = order_ppst_final30.loc[:, order_ppst_final30.columns == 'y_30']

le = LabelEncoder()
categorical_cols = ['cx_category','cx_gender','cx_channel','orderYear','orderMonth','TotalSegment','BBQPlazaDeli','BBQPlazaDineIn']
X30_real[categorical_cols] = X30_real[categorical_cols].apply(lambda col: le.fit_transform(col))

x30_real_scaled = min_max_scaler.fit_transform(X30_real)


/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/satayu.c/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [95]:
order_merge_wseg['probToCome_180']=logreg180.predict_proba(x180_real_scaled)[:,1]
order_merge_wseg['probToCome_90']=logreg90.predict_proba(x90_real_scaled)[:,1]
order_merge_wseg['probToCome_60']=logreg60.predict_proba(x60_real_scaled)[:,1]
order_merge_wseg['probToCome_30']=logreg30.predict_proba(x30_real_scaled)[:,1]


In [96]:
order_merge_wseg


,level_0,customers,cx_channel,branch,orderDate,pointBefore,pointEarn,pointUse,order_x,cx_totalPrice,...,Time,orderYear,orderMonth,TotalSegment,BBQPlazaDeli,BBQPlazaDineIn,probToCome_180,probToCome_90,probToCome_60,probToCome_30
0,1,2747641,Dine In,281,2023-01-29 19:15:45,97,62,0,31811149.0,799.0,...,1,2023,1,Promising,Unknown,Unknown,5.457532e-05,0.144904,0.240936,0.327131
1,2,2747641,Dine In,281,2023-03-07 20:15:55,159,44,0,32225415.0,1273.0,...,2,2023,3,Potential Loyalist,Unknown,Unknown,9.338385e-01,0.425419,0.408467,0.393410
2,2,2747641,Dine In,281,2023-03-07 20:15:55,159,44,0,32225415.0,1273.0,...,2,2023,3,Potential Loyalist,Unknown,Unknown,9.338385e-01,0.425419,0.408467,0.393410
3,6,2747645,Dine In,108,2022-12-22 14:19:00,185,44,0,30871775.0,1119.0,...,1,2022,12,Unknown,Unknown,Unknown,1.433500e-12,0.075427,0.234261,0.383867
4,7,2747645,Dine In,230,2023-02-25 16:42:38,229,62,0,32098346.0,1711.0,...,2,2023,2,Potential Loyalist,Unknown,Unknown,9.546547e-01,0.311412,0.364438,0.397607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820607,1940540,5638962,Dine In,255,2023-07-17 16:55:17,0,86,0,34105520.0,1446.0,...,1,2023,7,Unknown,Unknown,Unknown,1.000000e+00,0.945997,0.884980,0.717068
820608,1940741,5639240,Dine In,116,2023-07-17 18:43:52,0,132,0,34107488.0,2392.0,...,1,2023,7,Unknown,Unknown,Unknown,1.000000e+00,0.930266,0.855901,0.684113
820609,1940969,5639571,Dine In,228,2023-07-17 19:59:37,0,35,0,34109569.0,1048.0,...,1,2023,7,Unknown,Unknown,Unknown,1.000000e+00,0.915905,0.829369,0.625507
820610,1941001,5639621,Dine In,279,2023-07-17 20:20:00,0,258,0,34110209.0,3886.0,...,1,2023,7,Unknown,Unknown,Unknown,1.000000e+00,0.973655,0.932875,0.737706


In [ ]:
order_merge_wseg = order_merge_wseg[['customers','order_x','time','probToCome_180','probToCome_90','probToCome_60','probToCome_30']]
order_merge_wseg = order_merge_wseg.drop_duplicates()